# user_tweets

In [ ]:
modules = ["flair","spacy","spacytextblob"]

import sys
import os.path
from subprocess import check_call
import importlib
import os

def instala(modules):
  print("Instalando módulos")
  for m in modules:
      p = m.find("[")
      mi = m if p==-1 else m[:p]
      p = mi.find("==")
      mi = mi if p==-1 else mi[:p]

      torch_loader = importlib.util.find_spec(mi)
      if torch_loader is not None:
          print(m," encontrado")
      else:
          print(m," No encontrado, instalando...",end="")  
          try:        
            r = check_call([sys.executable, "-m", "pip", "install", "--user", m])
            print("¡hecho!")
          except:
            print("¡Problema al instalar ",m,"! ¿seguro que el módulo existe?",sep="")
              
  print("¡Terminado!")

instala(modules) 

In [ ]:
pipelines = ['en_core_web_lg']
def instala_pipelines(pipelines):
  print("Instalando pipelines de Spacy")
  for p in pipelines:
      print("Instalando ",p)
      try:
        r = check_call([sys.executable, "-m", "spacy", "download", p])
        print("¡Hecho!")
      except:
        print("¡Problema al instalar ",p,"! ¿seguro que el pipeline existe?",sep="")

  print("¡Terminado!")

instala_pipelines(pipelines)

In [ ]:
import pandas as pd
users_data_file = "users_common_data.csv"
user_file = "usa162024.csv"
edges_file = "aristas_p_y.csv"

path ="../data/"
import pandas as pd
df_user = pd.read_csv(user_file)

df_edges = pd.read_csv(edges_file)
# couple sorted alpahabetically, so that it is repeated
df_edges['pair_key'] = df_edges.apply(lambda row: tuple(sorted([row['user1'], row['user2']])), axis=1)
# eliminate duplicates from the sorted couple
df_edges = df_edges.drop_duplicates(subset='pair_key').drop(columns='pair_key').reset_index(drop=True)
df_edges.columns = ["user1", 	"user2", 	"weight", 	"party", "year"]
df_edges.weight  = df_edges.weight.apply(lambda x: round(x,3))

df_user_data = pd.read_csv(users_data_file)

url_opiniones = "opiniones.xlsx"

# load the spreadsheet in a pandas dataframe
df_opiniones = pd.read_csv(url_opiniones)

In [ ]:
df_user.head(3)

In [ ]:
df_user_data 

In [ ]:
df_edges

In [ ]:
from flair.models import TextClassifier
from flair.data import Sentence
classifier = TextClassifier.load('en-sentiment')

import spacy
from spacytextblob.spacytextblob import SpacyTextBlob
nlp = spacy.load('en_core_web_lg')
nlp.add_pipe('spacytextblob')

def sentiment(s):

  # flair
  text = Sentence(s)

  classifier.predict(text)
  value = text.labels[0].to_dict()
  #print(s)
  if value["value"]=="NEGATIVE":
      vflair = -1
  elif value["value"]=="POSITIVE":
      vflair = +1
  else :
      vflair = 0 # no sé si esto pasa
  cflair = value["confidence"]

  # textblob  
  return vflair,cflair #, vblob,cblob

In [ ]:
from IPython.display import display
def display_user(user,year,party):
    filter_user = (df_user.User == user) & (df_user.year==year) & (df_user.party==party)
    filter_edges = (df_edges.user1 == user) & (df_edges.year==year) & (df_edges.party==party)
    df_edges_filtered = df_edges[filter_edges]
    df_user_filtered = df_user[filter_user]
    
    df = df_user_filtered['Tweet'].apply(lambda tweet: pd.Series(sentiment(tweet)))
    df.columns = ['vflair', 'cflair'] #, 'vblob', 'cblob']
    print("="*50)
    print(user,year,party,end=" ")
    print(f"{len(df_edges_filtered)} connections, {len(df_user_filtered)} tweets")
    if len(df_user_filtered)>0:
        for tweet in df_user_filtered.Tweet:
            print(tweet)
            print("-"*50)
        #display(df)
        display(df.describe())
    return

def display_user_both(user,year):
    display_user(user,year,"dem")
    display_user(user,year,"rep")
    return

def display_user_all(user):
    for year in [16,20,24]:
        for party in ["dem","rep"]:
            display_user(user,year,party)
    return

In [ ]:
cols = ['User', 'year', 'party']
df_counts = df_user.groupby(cols).size().reset_index(name='reps')
print(df_counts.shape)
df = pd.merge(df_counts, df_opiniones, on=cols, how='outer', indicator=True)\
       .query("_merge != 'both'")\
       .drop('_merge', axis=1)\
       .reset_index(drop=True)\
       .sort_values(by=["reps"], ascending=False)
df[(df.year==24) & (df.party=="dem")].head(10)

In [ ]:
df_nedges = df_edges.groupby(["user1","party","year"]).size().reset_index(name='reps')
df_nedges.columns = ["User", "party", "year", "reps"]
df = pd.merge(df_nedges, df_opiniones, on=cols, how='outer', indicator=True)\
       .query("_merge != 'both'")\
       .drop('_merge', axis=1)\
       .reset_index(drop=True)\
       .sort_values(by=["reps"], ascending=False)
df[(df.year==24) & (df.party=="dem")].head(10)


Prueba con flair

In [ ]:
df_counts = df_user.groupby(['User', 'year', 'party']).size().reset_index(name='reps')
(df_counts.reps>=5).sum()

In [ ]:
df_activos = df_counts[(df_counts.reps>10)]
df_activos

In [ ]:
df_activos["opinion"] = 0
for p, row in df_activos.iterrows():
    filtro = (df_user.User == row["User"]) & (df_user.year == row["year"]) & (df_user.party==row["party"])
    df_user_filtered = df_user[filtro]
    df = df_user_filtered['Tweet'].apply(lambda tweet: pd.Series(sentiment(tweet)))
    df.columns = ['vflair', 'cflair']
    desc = df.describe()
    op  = abs(desc["vflair"]["mean"])
    c = abs(desc["cflair"]["mean"])
    if op>=0.5 and c>=0.8:
        print(".",end="")
        df_activos.loc[p,"opinion"] =  1 if desc["vflair"]["mean"]>0 else -1

    

In [ ]:
(df_activos.opinion !=0).sum()

In [ ]:
df_flair = df_activos[df_activos.opinion !=0].reset_index(drop=True)

In [ ]:
df_flair[df_flair.reps>10]

In [ ]:
path = "../data/"
df_flair.to_csv(path+"flair.csv",index=False)

In [ ]:
from IPython.display import display
def display_user(user,year,party):
    filter_user = (df_user.User == user) & (df_user.year==year) & (df_user.party==party)
    filter_edges = (df_edges.user1 == user) & (df_edges.year==year) & (df_edges.party==party)
    df_edges_filtered = df_edges[filter_edges]
    df_user_filtered = df_user[filter_user]
    
    
    print("="*50)
    print(user,year,party)
    print(f"{len(df_edges_filtered)} connections, {len(df_user_filtered)} tweets")
    for tweet in df_user_filtered.Tweet:
        print(tweet)
    return

In [ ]:
#sample = df_flair.sample(30)
df_opinion = pd.read_csv(path+"opiniones.csv")
for _,row in sample.iterrows():
    user,year,party = row["User"],row["year"],row["party"]
    # solo si es nuevo
    if len(df_opinion[(df_opinion.User==user) & (df_opinion.year==year) & (df_opinion.party==party)])==0:
        display_user(row["User"],row["year"],row["party"])


In [ ]:
sample


In [ ]:
# inconsistencias?
df_opinion = pd.read_csv(path+"opiniones.csv")
df_op_check = df_opinion.groupby(['User', 'year', 'party']).size().reset_index(name='reps')
df_op_check[df_op_check.reps>1]

In [ ]:
comb = df_flair.merge(df_opinion,left_on=['User', 'year', 'party'], right_on=['User', 'year', 'party'],suffixes=('_flair', '_manual'))
comb[comb["opinion_flair"] != comb["opinion_manual"]]

In [ ]:
df_flair

In [ ]:
df_flair["Autor"] = "flair"
df_flair = df_flair[["User","year","party","opinion","Autor"]]
df_flair

In [ ]:
cols_clave = ['User', 'year', 'party']

df_flair_filtrado = df_flair.merge(
    df_opinion[cols_clave],
    on=cols_clave,
    how='left',
    indicator=True
).query('_merge == "left_only"').drop(columns=['_merge'])


df_flair_filtrado

In [ ]:
df_opinion = pd.concat([df_opinion, df_flair],ignore_index=True)

In [ ]:
df_opinion.to_csv(path+"opiniones.csv",index=False)